In [ ]:
# https://www.python.org/dev/peps/pep-0249/
# https://docs.python.org/3/library/sqlite3.html
# https://pyneng.readthedocs.io/ru/latest/book/25_db/index.html

db_name = 'sqlite-api.db'

In [ ]:
import sqlite3
con = sqlite3.connect(db_name)

cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE IF NOT EXISTS stocks
               (date text, trans text, symbol text, qty real, price real)''')

try:
    # Insert a row of data
    cur.execute("INSERT INTO stocks VALUES ('2006-01-05', 'BUY', 'RHAT', 100, 35.14)")
    cur.execute("INSERT INTO stocks VALUES ('2006-03-28', 'BUY', 'IBM', 1000, 45.0)")
    cur.execute("INSERT INTO stocks VALUES ('2006-04-06', 'SELL', 'IBM', 500, 53.0)")
    cur.execute("INSERT INTO stocks VALUES ('2006-04-05', 'BUY', 'MSFT', 1000, 72.0)")


    # Save (commit) the changes
    con.commit()
finally:
    # We can also close the connection if we are done with it.
    # Just be sure any changes have been committed or they will be lost.
    con.close()

In [ ]:
import sqlite3
connnection = sqlite3.connect(db_name)
cursor = connnection.cursor()
try:
    for row in cursor.execute('SELECT * FROM stocks ORDER BY price'):
        print(row)

finally:
    connnection.close()        

In [ ]:
import sqlite3
connnection = sqlite3.connect(db_name)
cursor = connnection.cursor()
try:
    cursor.execute('SELECT * FROM stocks ORDER BY price')
    print(cursor.fetchone())    
    print(cursor.fetchone())    

finally:
    connnection.close()  

In [ ]:
import sqlite3
connnection = sqlite3.connect(db_name)
cursor = connnection.cursor()
try:
    cursor.execute('SELECT * FROM stocks ORDER BY price')
    while True:
        next_row = cursor.fetchone()
        if next_row:
            print(next_row)
        else:
            break

finally:
    connnection.close()  

In [ ]:
import sqlite3
connnection = sqlite3.connect(db_name)
cursor = connnection.cursor()
try:
    cursor.execute('SELECT * FROM stocks ORDER BY price')
    next_row = cursor.fetchmany(2)
    print(next_row)

finally:
    connnection.close() 

SQL operations usually need to use values from Python variables. 
However, beware of using Python’s string operations to assemble queries, as they are vulnerable to SQL injection attacks 
(see the xkcd webcomic for a humorous example of what can go wrong):


In [ ]:
import sqlite3
connnection = sqlite3.connect(db_name)
cursor = connnection.cursor()
try:
    symbol = 'RHAT'
    cursor.execute("SELECT * FROM stocks WHERE symbol = '%s'" % symbol)
    print(cursor.fetchone())
finally:
    connnection.close() 

Instead, use the DB-API’s parameter substitution. 
To insert a variable into a query string, use a placeholder in the string, and substitute the actual values into the query by providing them as a tuple of values to the second argument of the cursor’s execute() method. 

An SQL statement may use one of two kinds of placeholders: 
question marks (qmark style) or 
named placeholders (named style). 

For the qmark style, parameters must be a sequence. 
For the named style, it can be either a sequence or dict instance. 

The length of the sequence must match the number of placeholders, or a ProgrammingError is raised. 
If a dict is given, it must contain keys for all named parameters. 
Any extra items are ignored.

In [ ]:
import sqlite3
connnection = sqlite3.connect(db_name)
cursor = connnection.cursor()
try:
    cursor.execute("CREATE TABLE IF NOT EXISTS lang (name, first_appeared)")

    # This is the qmark style:
    cursor.execute("insert into lang values (?, ?)", ("C", 1972))

    # The qmark style used with executemany():
    lang_list = [
        ("Fortran", 1957),
        ("Python", 1991),
        ("Go", 2009),
    ]

    cursor.executemany("insert into lang values (?, ?)", lang_list)

    connnection.commit()

    # And this is the named style:
    cursor.execute("select * from lang where first_appeared=:year", {"year": 1972})
    print(cursor.fetchall())
finally:
    connnection.close() 
